In [10]:
# Make sure the source code auto reloads into the kernel
%load_ext autoreload
%autoreload 2

In [1]:
# To help preventing shared maemory errors
!ulimit -n 64000
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [2]:
#n permutations of train and validation sets for average PR results.
import random
import pickle
from train_and_eval import train, test
from Utils.logger import logger
websites = ['auto-aol','auto-yahoo']#['auto-aol','auto-yahoo','auto-motortrend','auto-autobytel', 'auto-msn' ]
attributes = ['model', 'price', 'engine', 'fuel_economy']
n=1
list_avg_pr = []
for itr in range(n):
    random.shuffle(websites)
    logger.info(f'Epoch: {itr+1}/{n}, web-sites: {websites}')
    val_websites, charDict, tagDict, model, n_classes = train(websites, attributes)
    logger.info(f'Training is finished, starting predicting')
    avg_pr_dict = test(val_websites, charDict, tagDict, model, n_classes)
    logger.info(f'Test predictions result: {avg_pr_dict}')
    list_avg_pr.append(avg_pr_dict)
    pickle.dump(list_avg_pr, open('./list_avg_pr.pkl', 'wb'))    

13:33:10 INFO (3548147748:12): Epoch: 1/1, web-sites: ['auto-aol', 'auto-yahoo']
13:33:10 INFO (train_and_eval:36): Loading ./data/English_charDict.pkl
13:33:10 INFO (train_and_eval:38): Dictionary ./data/English_charDict.pkl length: 85
13:33:10 INFO (train_and_eval:36): Loading ./data/HTMLTagDict.pkl
13:33:10 INFO (train_and_eval:38): Dictionary ./data/HTMLTagDict.pkl length: 39
13:33:10 INFO (train_and_eval:51): Instantiating the Model checkpoint.
13:33:10 INFO (train_and_eval:61): Instantiating the Sequential model


training websites - ['auto-aol']
validation websites - ['auto-yahoo']


13:33:15 INFO (train_and_eval:84): Instantiating the Training object


pretrained word vectors loaded - 400000


/Users/izapreev/miniforge3/envs/tfm/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:447: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
13:33:15 INFO (train_and_eval:87): Fitting the model

  | Name                    | Type             | Params
-------------------------------------------------------------
0 | charLevelWordEmbeddings | CharCNN          | 6.3 K 
1 | BiLSTM                  | BiLSTM           | 241 K 
2 | BiLSTM_xpath            | BiLSTM_xpath     | 12.2 K
3 | Leaf_embedding          | Embedding        | 1.2 K 
4 | pos_embedding           | Embedding        | 2.0 K 
5 | classifier              | Sequential       | 46.1 K
6 | loss               

...

Web pages:   0%|          | 0/2000 [00:00<?, ?it/s]

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


 218854 - nodes are loaded in swde_dataLoader_test


/Users/izapreev/miniforge3/envs/tfm/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/izapreev/miniforge3/envs/tfm/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
15:38:18 INFO (3548147748:16): Test predictions result: {1: (1.0, nan, nan), 2: (1.0, nan, nan), 3: (1.0, nan, nan), 4: (1.0, nan, nan)}


class - 1: precision = 1.0, recall = nan, F1 = nan
class - 2: precision = 1.0, recall = nan, F1 = nan
class - 3: precision = 1.0, recall = nan, F1 = nan
class - 4: precision = 1.0, recall = nan, F1 = nan


In [3]:
# To Create the training data for other verticals.

In [4]:
import pandas as pd
import pickle
import numpy
import os
from Utils.DOMTree import DOMTree

In [5]:
Datapath = './data' #directory path where each vertical's html pages from SWDE dataset (website wise folders)
vertical = 'auto'


In [6]:
#dataset processor: to find the set of fixed nodes for each of the websites
from DatasetCreation.storeFixedNodes import main
main(Datapath, vertical)


['auto-kbb(2000)', 'auto-autoweb(2000)', 'auto-aol(2000)', 'auto-yahoo(2000)', 'auto-motortrend(1267)', 'auto-autobytel(2000)', 'auto-carquotes(2000)', 'auto-cars(657)', 'auto-msn(2000)', 'auto-automotive(1999)']
auto-kbb 2000
163
auto-autoweb 2000
286
auto-aol 2000
517
auto-yahoo 2000
715
auto-motortrend 1267
932
auto-autobytel 2000
1130
auto-carquotes 2000
1239
auto-cars 657
1338
auto-msn 2000
1651
auto-automotive 1999
1973


In [7]:
#dataset processor: Store each website as pickle file
from DatasetCreation.storeAllTextNodes import main
main(Datapath, vertical)

['auto-kbb(2000)', 'auto-autoweb(2000)', 'auto-aol(2000)', 'auto-yahoo(2000)', 'auto-motortrend(1267)', 'auto-autobytel(2000)', 'auto-carquotes(2000)', 'auto-cars(657)', 'auto-msn(2000)', 'auto-automotive(1999)']
auto-kbb
avg no. of variable nodes = [113.325  149.8585]
auto-autoweb
avg no. of variable nodes = [ 42.697  117.3055]
auto-aol
avg no. of variable nodes = [108.082  215.0985]
auto-yahoo
avg no. of variable nodes = [109.427  182.2715]
auto-motortrend
avg no. of variable nodes = [ 25.23914759 204.31412786]
auto-autobytel
avg no. of variable nodes = [ 41.712 191.237]
auto-carquotes
avg no. of variable nodes = [39.8235 99.8315]
auto-cars
avg no. of variable nodes = [52.79604262 90.49467275]
auto-msn
avg no. of variable nodes = [ 95.1715 283.4425]
auto-automotive
avg no. of variable nodes = [ 77.32616308 298.83491746]


In [8]:
#dataset processor: append the friendCircle of each node in the DOMTree
from Utils.friendCircleExtractioin import main
main(Datapath, vertical, 5)

auto-kbb
average friend nodes - 9.361369733580359, partner nodes - 0.12720799769649557
auto-autoweb
average friend nodes - 9.178472910445214, partner nodes - 0.8818057195648655
auto-aol
average friend nodes - 8.862153685773654, partner nodes - 0.21809872754387077
auto-yahoo
average friend nodes - 9.035792462159163, partner nodes - 0.34364432964678576
auto-motortrend
average friend nodes - 5.614116323088254, partner nodes - 0.09452934092421043
auto-autobytel
average friend nodes - 9.51733666413649, partner nodes - 0.4654751145337193
auto-carquotes
average friend nodes - 8.938866956567063, partner nodes - 0.37580329100302323
auto-cars
average friend nodes - 9.429029676957922, partner nodes - 0.5336563050405798
auto-msn
average friend nodes - 9.205392640513809, partner nodes - 0.320845476101988
auto-automotive
average friend nodes - 8.29912641327332, partner nodes - 0.5069163978988117


In [9]:
#dataset processor: associate the ground truth label to each node
attributes = ['model', 'manufacturer', 'price']
from DatasetCreation.assignGroundTruthToEachNode import main
main(Datapath, vertical, attributes)

auto-kbb


FileNotFoundError: [Errno 2] No such file or directory: './data/groundTruth/auto/auto-kbb-model.txt'